# Optimize employee attendence planning via linear optimizion

We organize our attendence time in timeslots. So we have a set $T$ of timeslots and $T_W \subseteq T$ for workday timeslots and $T_F = T \setminus T_W$ for timeslots not during working hours. (Timeslots not during working hours are payed extra.) Timeslots $t_1$ and $t_2$ in $T$ are called consequitive if they are directly after each other (Example $t_1$ ends at 9am and $t_2$ starts at 9am). So if we write $t+1$ we mean the consequitive timeslot after $t$. We can only have one consequitive timeslot. When indexing timeslots $t_i$ 

We have a set of employees $E$ and we have a set of $E_N \subseteq E$ for normal employees also we have some employees who have more experience or more power called escalation employees $E_E \subseteq E$. Each employee can specify how many timeslots she/he aims to be allocated as a normal employee or an escalation employee. We call this mapping target workload for normal employees $twl_N: E_N \rightarrow \mathbb{N}$ and $twl_E: E_E \rightarrow \mathbb{N}$.
//TODO constraints for $twl_N$ and $twl_E$

Each employee can also specify in which timeslots they do not have time ($\bot$) and for which timeslots they have to be allocated ($\top$). They can also keep it unspecified ($\sim$) meaning that the algorithm is free to allocate the attendence. We call this function employee constraints $ec_{N}: E_N \times T \rightarrow \{ \top, \bot, \sim \}$ (normal employees) and $ec_{E}: E_E \times T \rightarrow \{ \top, \bot, \sim \}$ (for escalation employees).

In [1]:
import math
from ortools.sat.python import cp_model

amountTimeSlots = 4*4
timeSlots = range(amountTimeSlots)
timeSlotsWorkday = [1,2,5,6,9,10,13,14]
timeSlotsFree = [t for t in timeSlots if t not in timeSlotsWorkday]

amountEmployees = 5
employees = range(amountEmployees)
normalEmployees = [1,2,3,4]
escalationEmployees = [0,1,2]
targetWorkloadNormal = {}
for e in normalEmployees:
    targetWorkloadNormal[e] = 0
targetWorkloadEscalation = {}
for e in escalationEmployees:
    targetWorkloadEscalation[e] = 0
targetWorkloadEscalation[2] = 2
targetWorkloadEscalation[1] = 2
targetWorkloadEscalation[0] = 2

normalEmployeeConstraints = {}
for e in normalEmployees:
    for t in timeSlots:
        normalEmployeeConstraints[(e,t)] = None
normalEmployeeConstraints[(1,0)] = False
normalEmployeeConstraints[(2,0)] = False
escalationEmployeeConstraints = {}
for e in escalationEmployees:
    for t in timeSlots:
        escalationEmployeeConstraints[(e,t)] = None
escalationEmployeeConstraints[(2,0)] = True
escalationEmployeeConstraints[(0,1)] = False
escalationEmployeeConstraints[(1,1)] = False
escalationEmployeeConstraints[(0,2)] = False
escalationEmployeeConstraints[(1,2)] = False

## Model
We model the allocation of a normal employee to a timeslot with a set $A_N$ of predicates $\varphi_{e,t}$ $\Leftrightarrow$ $e$ has (normal) attendence at timeslot $t$
$$A_N = \{ \varphi_{e,t} \mid e \in E_N \wedge t \in T \} $$

Similary, we model the attendencies for escalation employees $\psi_{e,t}$ $\Leftrightarrow$ $e$ has (escaltaion) attendence at timeslot $t$
$$A_E = \{ \psi_{e,t} \mid e \in E_E \wedge t \in T \} $$

To sum how many timeslots of a given set of timeslots an employee is allocated, we use the following workload function for an allocation model $A$, an employee $e \in E$, and a set of timeslots $T_u \subseteq T$ ($btoi$-maps true to 1)
$$wl_A(T_u, e) = \sum_{t \in T_u}^\infty btoi( \varphi_{e,t} \in A \wedge \varphi_{e,t} )  $$

In [2]:
model = cp_model.CpModel()
    
assignedNormalTimeSlots = {}
for e in normalEmployees:
    for t in timeSlots:
        assignedNormalTimeSlots[(e,t)] = model.NewBoolVar("slot %i is for person %i" % (t,e))
        
assignedEscalationTimeSlots = {}
for e in escalationEmployees:
    for t in timeSlots:
        assignedEscalationTimeSlots[(e,t)] = model.NewBoolVar("escalation slot %i is for person %i" % (t,e))
        
def workload(inModel, inTimeSlots, employee):
    return sum(inModel[(employee,t)] for t in inTimeSlots)

## Rules
### Respect employee constraints
Normal employees should be allocated or not allocated according to their employee constraints $ec_N$ (Note, if two employees want the same timeslot we will not find any feasible solution):
$$\forall_{t \in T} \, \forall_{e \in E_N} \, ec_N(e,t) = \top \rightarrow \varphi_{e,t} $$
$$\forall_{t \in T} \, \forall_{e \in E_N} \, ec_N(e,t) = \bot \rightarrow \neg \varphi_{e,t} $$

Similary for escalation employees:
$$\forall_{t \in T} \, \forall_{e \in E_E} \, ec_E(e,t) = \top \rightarrow \psi_{e,t} $$
$$\forall_{t \in T} \, \forall_{e \in E_E} \, ec_E(e,t) = \bot \rightarrow \neg \psi_{e,t} $$

In [3]:
for e in normalEmployees:
    for t in timeSlots:
        if normalEmployeeConstraints[(e,t)] != None:
            model.Add(assignedNormalTimeSlots[(e,t)] == normalEmployeeConstraints[(e,t)])
            
for e in escalationEmployees:
    for t in timeSlots:
        if escalationEmployeeConstraints[(e,t)] != None:
            model.Add(assignedEscalationTimeSlots[(e,t)] == escalationEmployeeConstraints[(e,t)])

### Each timeslot is allocated with *one* normal *employee*:
$$ \forall_{t \in T} \, \exists_{e \in E_N} \, \varphi_{e,t} $$

For escalation:
$$ \forall_{t \in T} \, \exists_{e \in E_E} \, \psi_{e,t} $$

### Each timeslot is allocated with *not more than one* normal *employee*:
$$\forall_{t \in T} \, \forall_{e_1 \in E_N} \, \forall_{e_2 \in E_N} \, e_1 \neq e_2 \rightarrow ( \varphi_{e_1,t} \rightarrow \neg \varphi_{e_2,t})$$

For escalation:
$$\forall_{t \in T} \, \forall_{e_1 \in E_E} \, \forall_{e_2 \in E_E} \, e_1 \neq e_2 \rightarrow ( \psi_{e_1,t} \rightarrow \neg \psi_{e_2,t})$$

In [4]:
for t in timeSlots:
    model.Add(sum(assignedNormalTimeSlots[(e,t)] for e in normalEmployees) == 1)
    model.Add(sum(assignedEscalationTimeSlots[(e,t)] for e in escalationEmployees) == 1)

### No one can be his own escalation:
$$\forall_{t \in T} \, \forall_{e \in E_N \cap E_E} \, \varphi_{e,t} \rightarrow \neg \psi_{e,t} $$

In [5]:
for t in range(amountTimeSlots-1):
    for e in [e for e in escalationEmployees if e in normalEmployees]:
        model.AddImplication(assignedNormalTimeSlots[(e,t)], assignedEscalationTimeSlots[(e,t)].Not())

### Two *consecutive timeslots are not allocated with the same employee*:
$$\forall_{t \in T} \, \forall_{e \in E} \, \varphi_{e,t} \rightarrow \neg \varphi_{e,t+1}$$

In [6]:
for t in range(amountTimeSlots-1):
    for e in normalEmployees:
        model.AddImplication(assignedNormalTimeSlots[(e,t)], assignedNormalTimeSlots[(e,t+1)].Not())

## Optimizion
We build an *objective function* by associating costs to non optimal solutions. A non optimal solution is one in which an allocation is somehow not fair enough for an employee.

### All employees in $E_N$ should rotate the timeslots equally often in $A_N$
We partition the timeslots $T$ in groups of consequitive timeslots so that there is exactly one timeslot for each normal employee in $E_N$. We call the partitions rotations $R$. $i, j \in \mathbb{N}$:
$$R_i = todo $$
$$c_{\mbox{rotations}} = \sum_{i = 0}^{|R|} \sum_{e \in E_N } { |wl_{A_N}(R_i, e) - 1| }$$

The maximal cost (worst-case) of $c_{\mbox{rotations}}$ over all rotations will not exceed:
$$c_{\mbox{rotations}}^\mbox{max} = |R| * (2(|E_N| - 1)) $$

In [7]:
def calcAbsMinus(m1, m2): #helper
    m1IntVar = model.NewIntVar(-amountTimeSlots,amountTimeSlots, "absm1")
    m2IntVar = model.NewIntVar(-amountTimeSlots,amountTimeSlots, "absm2")
    absValueIntVar = model.NewIntVar(-amountTimeSlots,amountTimeSlots, "abs")
    model.Add(m1IntVar == m1 - m2)
    model.Add(m2IntVar == m2 - m1)
    model.AddMaxEquality(absValueIntVar, [m1IntVar, m2IntVar])
    return absValueIntVar

In [8]:
rotations = [timeSlots[i: i + len(normalEmployees)] for i in range(0, len(timeSlots), len(normalEmployees))]

costRotations = 0
costRotationPerEmployee = {}
for r in rotations:
    for e in normalEmployees:
        costRotationPerEmployee[e,r] = calcAbsMinus(workload(assignedNormalTimeSlots, r, e), 1)
        costRotations = costRotations + costRotationPerEmployee[e,r]
        
costRotationsMax = len(rotations) * 2 * (len(normalEmployees) - 1)

### All employees in $E_E$ should rotate the timeslots equally often in $A_E$
Again, we partition the timeslots $T$ in groups of consequitive timeslots so that there is exactly one timeslot for each escalation employee in $E_E$. We call the partitions rotations $R_E$. $i, j \in \mathbb{N}$:
$$R^i_E = todo $$
$$c_{\mbox{rotations-escalation}} = \sum_{i = 0}^{|R_E|} \sum_{e \in E_E } { |wl_{A_E}(R^i_E, e) - 1| }$$

The maximal cost (worst-case) of $c_{\mbox{rotations-escalation}}$ over all rotations will not exceed:
$$c_{\mbox{rotations-escalation}}^\mbox{max} = |R_E| * (2(|E_E| - 1)) $$

In [9]:
rotationsEscalation = [timeSlots[i: i + len(escalationEmployees)] for i in range(0, len(timeSlots), len(escalationEmployees))]

costRotationsEscalation = 0
costRotationEscalationPerEmployee = {}
for r in rotationsEscalation:
    for e in escalationEmployees:
        costRotationEscalationPerEmployee[e,r] = calcAbsMinus(workload(assignedEscalationTimeSlots, r, e), 1)
        costRotationsEscalation = costRotationsEscalation + costRotationEscalationPerEmployee[e,r]
        
costRotationsEscalationMax = len(rotationsEscalation) * 2 * (len(escalationEmployees) - 1)

### All employees in $E_E$ should be allocated as often as they targeted with $twl_E$ within $T_F$ in $A_E$
In an not linear setting we could define the cost like the following:
$$c_{\mbox{target-workload-escalation-1}} = \sum_{e \in E_E} (wl_{A_E}(T_F, e) - twl_E(e))^2$$

To keep things linear we seek to minimize the maximum difference between target workload and actual workload:
$$c_{\mbox{target-workload-escalation}} = max_{e \in E_E} \{ |wl_{A_E}(T_F, e) - twl_E(e)| \}$$
Unfortunately, we can run into the problem that someone specifies an infeasible high value for $twl_E$ so that the maximum difference will shadow the target workloads of all other employees. See following example:

| employee | $twl_E$ | $wl_{A_E}$ | diff |
|----------|---------|------------|-----:|
| $e_0$    | 100     | 10         | * 90 |
| $e_1$    | 0       | 10         | 10   |
| $e_2$    | 10      | 0          | 10   |

Employee $e_1$ does not want any workload and $e_2$ would be happy to take the 10 timeslots from $e_1$, but it will not be optimized because we are just looking at the maximum difference which is 90.

To reduce instances of this unwanted behaviour we require that $twl_E$ for any employee will not be unsatisfyable high. Thus, $twl_E$ for any employee can not exceed $|T_F|$ minus non working timeslots for employee constraints $ec_E$.

In [10]:
costTargetWorkloadEscalation = model.NewIntVar(0,len(timeSlotsFree), "costTargetWorkloadEscalation")
model.AddMaxEquality(costTargetWorkloadEscalation, [
        calcAbsMinus(workload(assignedEscalationTimeSlots, timeSlotsFree, e), targetWorkloadEscalation[e]) 
        for e in escalationEmployees])

We need to define $c_{\mbox{target-workload-escalation}}^\mbox{max}$ for the worst case or a simpler version which will always exceed the actual worst case:
$$c_{\mbox{target-workload-escalation}}^\mbox{max} = |T_F|$$

In [11]:
costTargetWorkloadEscalationMax = len(timeSlotsFree)

### All employees in $E_N$ should be allocated as often as they targeted with $twl_N$ within $T_F$
Here, we use similar techniques as for $c_{\mbox{target-workload-escalation}}$:
$$c_{\mbox{target-workload}} = max_{e \in E_N} \{ |wl_{A_N}(T_F, e) - twl_N(e)| \}$$

As noted above, we require $twl_N$ to be smaller than an infeasible target workload. Again we could take $|T_F|$ as a rough reference. The rules however will restrict such behaviour because after each timeslot someone else has to take over so a better limit is $\frac{|T_F|}{2}$.

In [12]:
costTargetWorkload = model.NewIntVar(0,len(timeSlotsFree), "costTargetWorkload")
model.AddMaxEquality(costTargetWorkload, [
        calcAbsMinus(workload(assignedNormalTimeSlots, timeSlotsFree, e), targetWorkloadNormal[e]) 
        for e in normalEmployees])

### Minimize costs
We have defined the objective function weighting the various components:
$$
\begin{aligned}
c_{\mbox{overall}} &= c_{\mbox{rotations-escalation}} \\
                   &+ c_{\mbox{rotations-escalation}}^\mbox{max} c_{\mbox{rotations}} \\
                   &+ c_{\mbox{rotations-escalation}}^\mbox{max} c_{\mbox{rotations}}^\mbox{max}  c_{\mbox{target-workload-escalation}} \\
                   &+ c_{\mbox{rotations-escalation}}^\mbox{max} c_{\mbox{rotations}}^\mbox{max} c_{\mbox{target-workload-escalation}}^\mbox{max} c_{\mbox{target-workload}}
\end{aligned}$$


In [13]:
costOverall = (costRotationsEscalation
               + costRotationsEscalationMax * costRotations 
               + costRotationsEscalationMax * costRotationsMax * costTargetWorkloadEscalation 
               + costRotationsEscalationMax * costRotationsMax * costTargetWorkloadEscalationMax * costTargetWorkload)
model.Minimize(costOverall)

In [14]:
solver = cp_model.CpSolver()
status = solver.StatusName(solver.Solve(model))
print(status)
    
for t in timeSlots:
    for e in normalEmployees:
        if solver.Value(assignedNormalTimeSlots[(e,t)]):
            if t in timeSlotsWorkday:
                print(str(assignedNormalTimeSlots[(e,t)]) + "*")
            else:
                print(assignedNormalTimeSlots[(e,t)])
                
for e in normalEmployees:
    print("workload %i: %s" % (e, str(solver.Value(workload(assignedNormalTimeSlots, timeSlots, e)))))
    
print("costRotations: %s" % str(solver.Value(costRotations)) )
print("costTargetWorkload: %s" % str(solver.Value(costTargetWorkload)) )

for r in rotations:
    for e in normalEmployees:
        print("cost for employee %i in rotation %s: %s" % (e, str(r), str(solver.Value(costRotationPerEmployee[(e,r)]))))
                
print("---------Now Escalations")                
for t in timeSlots:
    for e in escalationEmployees:
        if solver.Value(assignedEscalationTimeSlots[(e,t)]):
            if t in timeSlotsWorkday:
                print(str(assignedEscalationTimeSlots[(e,t)]) + "*")
            else:
                print(assignedEscalationTimeSlots[(e,t)])
                
print("costRotationsEscalation: %s" % str(solver.Value(costRotationsEscalation)) )
print("costTargetWorkloadEscalation %s" % str(solver.Value(costTargetWorkloadEscalation)))

for r in rotationsEscalation:
    for e in escalationEmployees:
        print("cost for employee %i in rotation %s: %s" % (e, str(r), str(solver.Value(costRotationEscalationPerEmployee[(e,r)]))))

OPTIMAL
slot 0 is for person 3
slot 1 is for person 1*
slot 2 is for person 4*
slot 3 is for person 2
slot 4 is for person 4
slot 5 is for person 3*
slot 6 is for person 2*
slot 7 is for person 1
slot 8 is for person 3
slot 9 is for person 1*
slot 10 is for person 2*
slot 11 is for person 4
slot 12 is for person 1
slot 13 is for person 3*
slot 14 is for person 4*
slot 15 is for person 2
workload 1: 4
workload 2: 4
workload 3: 4
workload 4: 4
costRotations: 0
costTargetWorkload: 2
cost for employee 1 in rotation range(0, 4): 0
cost for employee 2 in rotation range(0, 4): 0
cost for employee 3 in rotation range(0, 4): 0
cost for employee 4 in rotation range(0, 4): 0
cost for employee 1 in rotation range(4, 8): 0
cost for employee 2 in rotation range(4, 8): 0
cost for employee 3 in rotation range(4, 8): 0
cost for employee 4 in rotation range(4, 8): 0
cost for employee 1 in rotation range(8, 12): 0
cost for employee 2 in rotation range(8, 12): 0
cost for employee 3 in rotation range(8, 12